In [1]:
import os
import re
import json
import time
from pathlib import Path

os.environ["MPLBACKEND"] = "Agg"

import gym
import torch
import ale_py  # noqa: F401

from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack, VecTransposeImage, VecMonitor
from stable_baselines3.common.callbacks import BaseCallback, CallbackList


# =========================
# Callback: Heartbeat + log por episodio
# =========================
class LiveLoggerCallback(BaseCallback):
    def __init__(self, every_steps=100_000, verbose=1):
        super().__init__(verbose)
        self.every_steps = int(every_steps)
        self.t0 = None

    def _on_training_start(self) -> None:
        self.t0 = time.time()
        print("[live] training started", flush=True)

    def _on_step(self) -> bool:
        if self.num_timesteps % self.every_steps == 0:
            elapsed = time.time() - self.t0 if self.t0 else 0.0
            print(f"[heartbeat] timesteps={self.num_timesteps} elapsed_s={elapsed:.1f}", flush=True)

        #en VecEnv, infos es lista de dicts (uno por env)
        infos = self.locals.get("infos", [])
        for info in infos:
            ep = info.get("episode")
            if ep is None:
                continue
            ep_r = float(ep["r"])
            ep_l = int(ep["l"])
            print(f"[episode] r={ep_r:.1f} len={ep_l} | timesteps={self.num_timesteps}", flush=True)

        return True


# =========================
# Callback: objetivo 20+ por 100 episodios consecutivos (durante train)
# 
# =========================
class ConsecutiveRewardCallback(BaseCallback):
    def __init__(self, threshold=20.0, target_consecutive=100, save_path="model_reached_target", verbose=1):
        super().__init__(verbose)
        self.threshold = float(threshold)
        self.target_consecutive = int(target_consecutive)
        self.save_path = str(save_path)
        self.consecutive = 0
        self.best_consecutive = 0

    def _on_step(self) -> bool:
        infos = self.locals.get("infos", [])
        for info in infos:
            ep = info.get("episode")
            if ep is None:
                continue

            ep_r = float(ep["r"])

            if ep_r > self.threshold:
                self.consecutive += 1
            else:
                self.consecutive = 0

            self.best_consecutive = max(self.best_consecutive, self.consecutive)

            self.logger.record("rollout/consecutive_over_20", self.consecutive)
            self.logger.record("rollout/best_consecutive_over_20", self.best_consecutive)
            self.logger.record("rollout/episode_reward", ep_r)

            if self.verbose and (self.consecutive == 1 or self.consecutive % 10 == 0):
                print(
                    f"[racha] ep_reward={ep_r:.1f} | consecutive>{self.threshold}={self.consecutive} "
                    f"(best={self.best_consecutive})",
                    flush=True
                )

           
            if self.consecutive >= self.target_consecutive:
                print(f"\n OBJETIVO CUMPLIDO (TRAIN): {self.consecutive} episodios seguidos > {self.threshold}", flush=True)
                self.model.save(self.save_path)
                return False

        return True


# =========================
# Callback: save modelo + replay buffer + metadata (SB3 1.8.0 safe)
# =========================
class SaveModelBufferAndMetaCallback(BaseCallback):
    def __init__(
        self,
        ckpt_dir,
        name_prefix="dqn_si",
        save_freq=100_000,
        buffer_freq=500_000,
        meta_freq=25_000,
        save_replay_buffer=True,
        run_config=None,
        verbose=1
    ):
        super().__init__(verbose)
        self.ckpt_dir = Path(ckpt_dir)
        self.ckpt_dir.mkdir(parents=True, exist_ok=True)

        self.name_prefix = str(name_prefix)
        self.save_freq = int(save_freq)
        self.buffer_freq = int(buffer_freq)
        self.meta_freq = int(meta_freq)
        self.save_replay_buffer = bool(save_replay_buffer)

        self.run_config = run_config or {}

        self.buffer_path = self.ckpt_dir / f"{self.name_prefix}_replay_buffer.pkl"
        self.meta_path = self.ckpt_dir / f"{self.name_prefix}_meta.json"

    def _atomic_write_text(self, path: Path, text: str):
        tmp = path.with_suffix(path.suffix + ".tmp")
        tmp.write_text(text, encoding="utf-8")
        tmp.replace(path)

    def _save_meta(self):
        meta = dict(self.run_config)
        meta.update({
            "num_timesteps": int(self.num_timesteps),
            "timestamp": time.time(),
        })
        if hasattr(self.model, "exploration_rate"):
            meta["exploration_rate"] = float(self.model.exploration_rate)

        self._atomic_write_text(self.meta_path, json.dumps(meta, indent=2))
        if self.verbose:
            print(f"[meta] saved: {self.meta_path}", flush=True)

    def _on_step(self) -> bool:
        # 1) modelo numerado
        if self.num_timesteps > 0 and self.num_timesteps % self.save_freq == 0:
            model_path = self.ckpt_dir / f"{self.name_prefix}_{self.num_timesteps}_steps"
            self.model.save(str(model_path))
            if self.verbose:
                print(f"[ckpt] saved model: {model_path}.zip", flush=True)

        # 2) replay buffer overwrite
        if self.save_replay_buffer and self.num_timesteps > 0 and self.num_timesteps % self.buffer_freq == 0:
            if hasattr(self.model, "save_replay_buffer"):
                self.model.save_replay_buffer(str(self.buffer_path))
                if self.verbose:
                    print(f"[buffer] saved replay buffer (overwrite): {self.buffer_path}", flush=True)

        # 3) metadata
        if self.num_timesteps > 0 and self.num_timesteps % self.meta_freq == 0:
            self._save_meta()

        return True

    def _on_training_end(self) -> None:
        try:
            self._save_meta()
        except Exception as e:
            print(f"[meta] warning: could not save meta on end: {e}", flush=True)


# =========================
# Utils resume
# =========================
_ckpt_re = re.compile(r".*_(\d+)_steps\.zip$")

def find_latest_checkpoint(ckpt_dir: Path, prefix="dqn_si"):
    ckpts = sorted(ckpt_dir.glob(f"{prefix}_*_steps.zip"))
    best = None
    best_steps = -1
    for p in ckpts:
        m = _ckpt_re.match(str(p))
        if not m:
            continue
        steps = int(m.group(1))
        if steps > best_steps:
            best_steps = steps
            best = p
    return best, best_steps

def load_meta(meta_path: Path):
    if meta_path.exists():
        try:
            return json.loads(meta_path.read_text(encoding="utf-8"))
        except Exception:
            return None
    return None


def main():
    print("Gym:", gym.__version__)
    print("Torch:", torch.__version__, "CUDA:", torch.cuda.is_available(), flush=True)

    # ====== Config ======
    env_id = "ALE/SpaceInvaders-v5"
    n_envs = 4
    seed = 123
    total_target_timesteps = 20_000_000  

    base_dir = Path(os.getcwd())
    ckpt_dir = base_dir / "checkpoints"
    log_dir = base_dir / "logs_tensorboard"
    ckpt_dir.mkdir(exist_ok=True)
    log_dir.mkdir(exist_ok=True)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f" Paths:\n  ckpt_dir={ckpt_dir}\n  log_dir={log_dir}\n  device={device}", flush=True)

    # ====== Env (igual que entrenamiento anterior) ======
    env = make_atari_env(
        env_id,
        n_envs=n_envs,
        seed=seed,
        wrapper_kwargs=dict(clip_reward=True, noop_max=0),
    )
    env = VecMonitor(env)
    env = VecFrameStack(env, n_stack=4)
    env = VecTransposeImage(env)

    # ====== Callbacks ======
    run_config = {"env_id": env_id, "n_envs": n_envs, "seed": seed, "total_target_timesteps": total_target_timesteps}

    save_cb = SaveModelBufferAndMetaCallback(
        ckpt_dir=ckpt_dir,
        name_prefix="dqn_si",
        save_freq=25_000,
        buffer_freq=100_000,
        meta_freq=25_000,
        save_replay_buffer=True,
        run_config=run_config,
        verbose=1
    )
    live_cb = LiveLoggerCallback(every_steps=100_000, verbose=1)
    goal_cb = ConsecutiveRewardCallback(threshold=20.0, target_consecutive=100,
                                        save_path=ckpt_dir / "dqn_si_reached_20x100", verbose=1)

    callbacks = CallbackList([save_cb, live_cb, goal_cb])

    # ====== Resume ======
    latest, latest_steps = find_latest_checkpoint(ckpt_dir, prefix="dqn_si")
    meta = load_meta(ckpt_dir / "dqn_si_meta.json")

    if latest is not None:
        print(f"Reanudando desde: {latest} (steps={latest_steps})", flush=True)
        model = DQN.load(str(latest), env=env, device=device)

        # Cargar replay buffer
        rb_path = ckpt_dir / "dqn_si_replay_buffer.pkl"
        if rb_path.exists():
            try:
                model.load_replay_buffer(str(rb_path))
                print(f"Replay buffer cargado: {rb_path}", flush=True)
            except Exception as e:
                print(f"No pude cargar replay buffer: {e}", flush=True)

        # Restaurar epsilon si existe
        if meta and "exploration_rate" in meta and hasattr(model, "exploration_rate"):
            try:
                model.exploration_rate = float(meta["exploration_rate"])
                print(f"exploration_rate restaurado: {model.exploration_rate}", flush=True)
            except Exception as e:
                print(f" No pude restaurar exploration_rate: {e}", flush=True)

        reset_num_timesteps = False
        remaining = max(0, total_target_timesteps - latest_steps)
        print(f"Timesteps restantes: {remaining} (target={total_target_timesteps})", flush=True)

    else:
        print("No hay checkpoints. Entrenando desde cero.", flush=True)
        reset_num_timesteps = True
        remaining = total_target_timesteps

        # SB3 1.8.0: NO replay_buffer_kwargs, mejor optimize_memory_usage=False
        model = DQN(
            "CnnPolicy",
            env,
            learning_rate=1e-4,
            buffer_size=400_000,
            learning_starts=10_000,
            batch_size=64,
            gamma=0.99,
            train_freq=4,
            gradient_steps=1,
            target_update_interval=10_000,
            exploration_fraction=0.25,
            exploration_initial_eps=1.0,
            exploration_final_eps=0.01,
            optimize_memory_usage=False,
            tensorboard_log=str(log_dir),
            verbose=1,
            device=device,
        )

    # ====== Train ======
    if remaining > 0:
        model.learn(
            total_timesteps=remaining,
            callback=callbacks,
            reset_num_timesteps=reset_num_timesteps
        )
    else:
        print("Esta en/por encima del target; no se entrena más.", flush=True)

    # Guardado final
    model.save(str(ckpt_dir / "dqn_si_last"))
    if hasattr(model, "save_replay_buffer"):
        model.save_replay_buffer(str(ckpt_dir / "dqn_si_replay_buffer.pkl"))

    env.close()
    print(" Entrenamiento terminado. Modelo guardado.", flush=True)


if __name__ == "__main__":
    main()


Gym: 0.21.0
Torch: 1.13.1+cu117 CUDA: True
 Paths:
  ckpt_dir=C:\08MIAR\LOCAL\checkpoints
  log_dir=C:\08MIAR\LOCAL\logs_tensorboard
  device=cuda
Reanudando desde: C:\08MIAR\LOCAL\checkpoints\dqn_si_19925000_steps.zip (steps=19925000)


C:\Users\aurno\anaconda3\envs\gym021\lib\site-packages\stable_baselines3\common\vec_env\vec_monitor.py:44: UserWarning: The environment is already wrapped with a `Monitor` wrapperbut you are wrapping it with a `VecMonitor` wrapper, the `Monitor` statistics will beoverwritten by the `VecMonitor` ones.
  warnings.warn(
C:\Users\aurno\anaconda3\envs\gym021\lib\site-packages\stable_baselines3\common\buffers.py:229: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 22.59GB > 21.31GB
  warnings.warn(


Replay buffer cargado: C:\08MIAR\LOCAL\checkpoints\dqn_si_replay_buffer.pkl
exploration_rate restaurado: 0.01
Timesteps restantes: 75000 (target=20000000)
Logging to C:\08MIAR\LOCAL\logs_tensorboard\DQN_1
[live] training started
[episode] r=31.0 len=104 | timesteps=19925416
[racha] ep_reward=31.0 | consecutive>20.0=1 (best=1)
[episode] r=32.0 len=105 | timesteps=19925420
[episode] r=1.0 len=18 | timesteps=19925492
[racha] ep_reward=1.0 | consecutive>20.0=0 (best=2)
[episode] r=39.0 len=142 | timesteps=19925568
[racha] ep_reward=39.0 | consecutive>20.0=1 (best=2)
------------------------------------------
| rollout/                    |          |
|    best_consecutive_over_20 | 2        |
|    consecutive_over_20      | 1        |
|    ep_len_mean              | 90.2     |
|    ep_rew_mean              | 23.09    |
|    episode_reward           | 39       |
|    exploration_rate         | 0.01     |
| time/                       |          |
|    episodes                 | 271124   |
|

# VALIDACION

In [2]:
import os
import time
from pathlib import Path

os.environ["MPLBACKEND"] = "Agg"

import gym
import torch
import ale_py  # noqa: F401  (registra ALE envs)

from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack, VecTransposeImage, VecMonitor


def _reset_compat(env):
    """
    Compatibilidad: algunas APIs devuelven obs o (obs, info).
    VecEnv normalmente devuelve obs directo, pero lo dejamos robusto.
    """
    out = env.reset()
    if isinstance(out, tuple) and len(out) == 2:
        return out[0]
    return out


def _step_compat(env, action):
    """
    Compatibilidad:
    - Gym clásico: obs, reward, done, info
    - Gymnasium: obs, reward, terminated, truncated, info
    - VecEnv: reward/done vienen como arrays de tamaño n_envs
    """
    out = env.step(action)
    if len(out) == 4:
        obs, reward, done, info = out
        terminated = done
        truncated = [False] if hasattr(done, "__len__") else False
        return obs, reward, terminated, truncated, info
    elif len(out) == 5:
        obs, reward, terminated, truncated, info = out
        return obs, reward, terminated, truncated, info
    else:
        raise RuntimeError(f"step() devolvió {len(out)} elementos, no esperado.")


def main():
    # ====== Ajustes ======
    base_dir = Path(os.getcwd())
    model_path = base_dir / "checkpoints" / "dqn_si_last.zip"

    env_id = "ALE/SpaceInvaders-v5"
    seed = 123

    # Criterio de éxito:
    threshold = 20.0
    target_consecutive = 100

    # Límite de seguridad:
    max_episodes = 50_000

    # Logs:
    print_every_episodes = 10        # imprime cada N episodios
    heartbeat_seconds = 30           # imprime “sigo vivo” cada N segundos aunque no termine episodio
    print_episode_debug = False      # pon True solo si estás debuggeando 10-50 episodios

    # ====== Verificaciones ======
    print("Gym:", gym.__version__)
    print("Torch:", torch.__version__, "CUDA:", torch.cuda.is_available(), flush=True)

    if not model_path.exists():
        raise FileNotFoundError(f"No encontré el modelo: {model_path}")

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f" Cargando modelo: {model_path} (device={device})", flush=True)

    # ====== Env (RECOMENDADO): usar configuración estándar (sticky actions por defecto)
    # Importante: NO forzamos repeat_action_probability=0.0 aquí,
    # para que sea lo más “oficial”/estándar posible.
    env = make_atari_env(
        env_id,
        n_envs=1,
        seed=seed,
        wrapper_kwargs=dict(
            clip_reward=True,  # reward clipping activo
            noop_max=0,        # evita bug randint vs Generator en wrappers antiguos
        ),
    )

    env = VecMonitor(env)             # para info["episode"] consistente
    env = VecFrameStack(env, n_stack=4)
    env = VecTransposeImage(env)

    # ====== Modelo (EVITA reservar replay buffer gigante al cargar) ======
    # En SB3 1.8.0, al cargar reconstruye el ReplayBuffer usando buffer_size del entrenamiento,
    # lo que puede causar MemoryError. Forzamos valores mínimos para evaluación.
    model = DQN.load(
        str(model_path),
        env=env,
        device=device,
        custom_objects={
            "buffer_size": 1,               # no reservamos 100s de miles
            "learning_starts": 0,
            "optimize_memory_usage": False
        }
    )

    # ====== Evaluación determinística ======
    consecutive = 0
    best_consecutive = 0
    episodes = 0
    last_hb = time.time()

    obs = _reset_compat(env)

    # Acumulador por episodio (backup si no llega info["episode"])
    ep_return = 0.0
    ep_len = 0

    print("[eval] start (deterministic=True)", flush=True)

    while episodes < max_episodes and consecutive < target_consecutive:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, infos = _step_compat(env, action)

        # VecEnv: reward suele ser array([r])
        r = float(reward[0]) if hasattr(reward, "__len__") else float(reward)
        ep_return += r
        ep_len += 1

        done = bool(terminated[0]) if hasattr(terminated, "__len__") else bool(terminated)
        trunc = bool(truncated[0]) if hasattr(truncated, "__len__") else bool(truncated)

        # Heartbeat temporal (aunque no termine episodio)
        now = time.time()
        if now - last_hb >= heartbeat_seconds:
            print(f"[heartbeat] episodes={episodes} best={best_consecutive} current={consecutive}", flush=True)
            last_hb = now

        if done or trunc:
            episodes += 1

            info0 = infos[0] if isinstance(infos, (list, tuple)) else infos
            ep = info0.get("episode") if isinstance(info0, dict) else None

            if ep is not None and "r" in ep and "l" in ep:
                ep_r = float(ep["r"])
                ep_l = int(ep["l"])
                src = "monitor"
            else:
                ep_r = float(ep_return)
                ep_l = int(ep_len)
                src = "manual"

            # Criterio de racha
            if ep_r > threshold:
                consecutive += 1
            else:
                consecutive = 0
            best_consecutive = max(best_consecutive, consecutive)

            # Logs (controlados)
            if print_episode_debug:
                tl = info0.get("TimeLimit.truncated") if isinstance(info0, dict) else None
                print(f"done={done} trunc={trunc} TimeLimit={tl}", flush=True)
                print(f"[episode {episodes}] src={src} r={ep_r:.1f} len={ep_l}", flush=True)

            if episodes % print_every_episodes == 0 or ep_r > threshold:
                print(
                    f"[episode {episodes}] r={ep_r:.1f} len={ep_l} | "
                    f"consecutive>{threshold}={consecutive} (best={best_consecutive})",
                    flush=True
                )

            # Reset acumuladores
            ep_return = 0.0
            ep_len = 0

            # Reset env
            obs = _reset_compat(env)

    env.close()

    if consecutive >= target_consecutive:
        print(
            f"\n OBJETIVO CUMPLIDO EN EVALUACIÓN: {consecutive} episodios seguidos con reward(clipped) > {threshold}",
            flush=True
        )
        print(f"   Episodios totales usados: {episodes}", flush=True)
    else:
        print(
            f"\n No se logró la racha. best_consecutive={best_consecutive} "
            f"en {episodes} episodios (límite={max_episodes}).",
            flush=True
        )


if __name__ == "__main__":
    main()


Gym: 0.21.0
Torch: 1.13.1+cu117 CUDA: True
 Cargando modelo: C:\08MIAR\LOCAL\checkpoints\dqn_si_last.zip (device=cuda)
[eval] start (deterministic=True)
[episode 1] r=35.0 len=133 | consecutive>20.0=1 (best=1)
[episode 3] r=23.0 len=68 | consecutive>20.0=1 (best=1)
[episode 4] r=52.0 len=193 | consecutive>20.0=2 (best=2)
[episode 5] r=29.0 len=101 | consecutive>20.0=3 (best=3)
[episode 6] r=23.0 len=64 | consecutive>20.0=4 (best=4)
[episode 7] r=36.0 len=130 | consecutive>20.0=5 (best=5)
[episode 9] r=43.0 len=141 | consecutive>20.0=1 (best=5)
[episode 10] r=33.0 len=112 | consecutive>20.0=2 (best=5)
[episode 11] r=21.0 len=81 | consecutive>20.0=3 (best=5)
[episode 12] r=25.0 len=136 | consecutive>20.0=4 (best=5)
[episode 13] r=33.0 len=113 | consecutive>20.0=5 (best=5)
[episode 15] r=60.0 len=223 | consecutive>20.0=1 (best=5)
[episode 16] r=76.0 len=268 | consecutive>20.0=2 (best=5)
[episode 20] r=20.0 len=68 | consecutive>20.0=0 (best=5)
[episode 30] r=9.0 len=67 | consecutive>20.0=0

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[episode 16951] r=68.0 len=230 | consecutive>20.0=1 (best=10)
[episode 16952] r=57.0 len=193 | consecutive>20.0=2 (best=10)
[episode 16957] r=62.0 len=213 | consecutive>20.0=1 (best=10)
[episode 16960] r=66.0 len=225 | consecutive>20.0=1 (best=10)
[heartbeat] episodes=16961 best=10 current=0
[episode 16962] r=32.0 len=140 | consecutive>20.0=1 (best=10)
[episode 16963] r=24.0 len=78 | consecutive>20.0=2 (best=10)
[episode 16966] r=48.0 len=173 | consecutive>20.0=1 (best=10)
[episode 16967] r=73.0 len=245 | consecutive>20.0=2 (best=10)
[episode 16969] r=50.0 len=172 | consecutive>20.0=1 (best=10)
[episode 16970] r=18.0 len=73 | consecutive>20.0=0 (best=10)
[episode 16972] r=33.0 len=115 | consecutive>20.0=1 (best=10)
[episode 16974] r=21.0 len=61 | consecutive>20.0=1 (best=10)
[episode 16975] r=39.0 len=143 | consecutive>20.0=2 (best=10)
[episode 16976] r=36.0 len=133 | consecutive>20.0=3 (best=10)
[episode 16978] r=87.0 len=295 | consecutive>20.0=1 (best=10)
[episode 16980] r=6.0 len=30

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

